In [1]:
import numpy as np
import pandas as pd

In [2]:
import cv2
import numpy as np
import os
import PIL

In [15]:
color_images = "C://Users//hp//Desktop//Research_intern_iitd//color_new"
color_crop =  "C://Users//hp//Desktop//Research_intern_iitd//color_crop"
config = "C://Users//hp//Desktop//Research_intern_iitd/yolo//config.cfg"
weights =  "C://Users//hp//Desktop//Research_intern_iitd/yolo//weightss.weights"
classes = ['fish']

In [4]:
def load_color(folder):
    images = []
    for filename in os.listdir(folder):
        #img = os.path.join(folder,filename)
        img = folder+'//'+filename
        if img is not None:
            images.append(img)
    return images

In [5]:
colorimage = load_color(color_images)

In [6]:
print(colorimage[0])

C://Users//hp//Desktop//Research_intern_iitd//color_new//0000.jpg


In [7]:
cnt = 0
for im_path in colorimage:
    #full_im = cv2.imread(im_path)
    #crop_im = full_im[290:660,600:1640]
    x = cnt
    suffix=str(x).zfill(4)
    #print(suffix)
    #cv2.imwrite("C:/Users/hp/Desktop/Research_intern_iitd/color_crop/{}.jpg".format(suffix),crop_im)
    cnt+=1

In [8]:
full_im = cv2.imread(color_images)
crop_im = full_im[290:660,600:1640]
cv2.imshow("cropped", crop_im)
cv2.waitKey(0)

TypeError: 'NoneType' object is not subscriptable

In [9]:
cropimage = load_color(color_crop)

In [10]:
cropimage[0]

'C://Users//hp//Desktop//Research_intern_iitd//color_crop//0000.jpg'

In [11]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [27]:
fish = {'fish1':{'raw_x':[],'raw_y':[],'width':[],'height':[]},
        'fish2':{'raw_x':[],'raw_y':[],'width':[],'height':[]},
        'fish3':{'raw_x':[],'raw_y':[],'width':[],'height':[]},
        'fish4':{'raw_x':[],'raw_y':[],'width':[],'height':[]},
        'fish5':{'raw_x':[],'raw_y':[],'width':[],'height':[]},
        'fish6':{'raw_x':[],'raw_y':[],'width':[],'height':[]},
        'fish7':{'raw_x':[],'raw_y':[],'width':[],'height':[]},
       }

def func(cnt):
    x = cnt+1
    res = "fish" + str(x)
    return res

In [28]:
cnt = 0
pboxes = []
pindices = None
for impath in cropimage:    
    image = cv2.imread(impath)
    image = cv2.resize(image,(416,416))
    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392

    # read class names from text file
    """classes = None
    with open(args.classes, 'r') as f:
        classes = [line.strip() for line in f.readlines()]"""

    # generate different colors for different classes 
    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

    # read pre-trained model and config file
    net = cv2.dnn.readNet(weights, config)

    # create input blob 

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

    # set input blob for the network
    net.setInput(blob)

    net
    layer_names = net.getLayerNames()

    # run inference through the network
    # and gather predictions from output layers
    outs = net.forward(get_output_layers(net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.2
    nms_threshold = 0.4

    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # go through the detections remaining
    # after nms and draw bounding box
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    #print("success ",cnt)
    #cnt+=1    
    if(cnt==0):
        pboxes = boxes
        pindices = indices
        ginti = 0
        for i in indices:
            print("Zero Loop : ", i)
            i = i[0]
            box = boxes[i]
            fish[func(ginti)]['raw_x'].append(box[0])
            fish[func(ginti)]['raw_y'].append(box[1])
            fish[func(ginti)]['width'].append(box[2])
            fish[func(ginti)]['height'].append(box[3])
    else:
        for i in indices:
            print("outer loop : ",i[0])
            i = i[0]
            box = boxes[i]
            cx = box[0]
            cy = box[1]
            cw = box[2]
            ch = box[3]
            mindiff = 1e18
            match = i
            jagah = 0
            choti_ginti = 0
            for j in pindices:
                print("inner loop : ",j[0])
                j = j[0]
                box2 = pboxes[j]
                px = box2[0]
                py = box2[1]
                pw = box2[2]
                ph = box2[3]
                curdiff = abs(px-cx)+abs(py-cy)+(abs(ph*pw - ch*cw)/ph*pw)
                if(curdiff<mindiff):
                    mindiff = curdiff
                    match = j
                    jagah = choti_ginti
                choti_ginti+=1
            fish[func(jagah)]['raw_x'].append(cx)
            fish[func(jagah)]['raw_y'].append(cy)
            fish[func(jagah)]['width'].append(cw)
            fish[func(jagah)]['height'].append(ch)
        pindices = indices
        pboxes = boxes
    x = cnt
    suffix = str(x).zfill(4)
    #cv2.imwrite("C://Users//hp//Desktop//Research_intern_iitd/color_detected/{}.jpg".format(suffix), image)
    
    cnt+=1
    if(cnt==10):
        break
        
df1 = pd.DataFrame(fish['fish1'])
df1

Zero Loop :  [7]
Zero Loop :  [3]
Zero Loop :  [1]
Zero Loop :  [0]
outer loop :  11
inner loop :  7
inner loop :  3
inner loop :  1
inner loop :  0
outer loop :  6
inner loop :  7
inner loop :  3
inner loop :  1
inner loop :  0
outer loop :  0
inner loop :  7
inner loop :  3
inner loop :  1
inner loop :  0
outer loop :  8
inner loop :  7
inner loop :  3
inner loop :  1
inner loop :  0
outer loop :  13
inner loop :  7
inner loop :  3
inner loop :  1
inner loop :  0
outer loop :  11
inner loop :  11
inner loop :  6
inner loop :  0
inner loop :  8
inner loop :  13
outer loop :  6
inner loop :  11
inner loop :  6
inner loop :  0
inner loop :  8
inner loop :  13
outer loop :  0
inner loop :  11
inner loop :  6
inner loop :  0
inner loop :  8
inner loop :  13
outer loop :  8
inner loop :  11
inner loop :  6
inner loop :  0
inner loop :  8
inner loop :  13
outer loop :  8
inner loop :  11
inner loop :  6
inner loop :  0
inner loop :  8
outer loop :  5
inner loop :  11
inner loop :  6
inner l

,raw_x,raw_y,width,height
0,38.0,332.0,62,46
1,165.5,316.0,35,50
2,353.5,310.5,53,47
3,363.0,46.0,28,84
4,38.5,333.0,63,46
5,38.0,332.5,64,47
6,37.0,331.5,62,49
7,37.5,331.0,61,50
8,37.0,332.0,62,48
9,37.0,334.0,62,44
